# Simple Example

This notebook demonstrates how to model a simple energy system with **flixopt**, including:

- Multiple energy carriers (electricity, heat, gas)
- A Combined Heat and Power (CHP) unit
- Thermal storage
- Effects for costs and CO2 emissions

This example provides a solid foundation for understanding flixopt's core concepts.

## Setup

In [1]:
import numpy as np
import pandas as pd

import flixopt as fx

# Configure for notebook environment
fx.CONFIG.notebook()

flixopt.config.CONFIG

## Create Time Series Data

First, we define the input data for our model:
- **Heat demand**: The thermal load profile over time (in kW)
- **Power prices**: Electricity prices for feed-in revenue (in €/kWh)

In [2]:
# Heat demand profile (kW) - varies throughout the day
heat_demand_per_h = np.array([30, 0, 90, 110, 110, 20, 20, 20, 20])

# Electricity prices (€/kWh) - constant for simplicity
power_prices = 1 / 1000 * np.array([80, 80, 80, 80, 80, 80, 80, 80, 80])

# Create datetime index
timesteps = pd.date_range('2020-01-01', periods=len(heat_demand_per_h), freq='h')

print(f'Optimization horizon: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

Optimization horizon: 2020-01-01 00:00:00 to 2020-01-01 08:00:00
Number of timesteps: 9


## Create FlowSystem

The `FlowSystem` is the main container that holds all components and manages the optimization.

In [3]:
flow_system = fx.FlowSystem(timesteps=timesteps)

## Define Energy Buses

**Buses** are nodes where energy carriers are balanced. The fundamental constraint is:

$$\sum \text{inputs} = \sum \text{outputs}$$

We use **carriers** to automatically assign colors in plots:
- `electricity` → yellow
- `heat` → red  
- `gas` → blue

In [4]:
flow_system.add_elements(
    fx.Bus(label='Strom', carrier='electricity'),
    fx.Bus(label='Fernwärme', carrier='heat'),
    fx.Bus(label='Gas', carrier='gas'),
)

2025-12-08 14:25:56.993 INFO     │ Registered new Bus: Strom
2025-12-08 14:25:56.994 INFO     │ Registered new Bus: Fernwärme
2025-12-08 14:25:56.994 INFO     │ Registered new Bus: Gas


## Define Effects

**Effects** are quantities that can be:
- **Tracked** - accumulated over the optimization horizon
- **Constrained** - limited by minimum/maximum values
- **Optimized** - used as the objective function

Here we define:
1. **Costs** - The objective to minimize (with CO2 price linkage)
2. **CO2** - Emissions with an hourly limit

In [5]:
# Cost effect - used as the optimization objective
costs = fx.Effect(
    label='costs',
    unit='€',
    description='Kosten',
    is_standard=True,  # Costs are automatically assigned if no effect is specified
    is_objective=True,  # Minimize this effect
    share_from_temporal={'CO2': 0.2},  # Add 0.2 €/kg CO2 as carbon price
)

# CO2 emissions effect with hourly constraint
CO2 = fx.Effect(
    label='CO2',
    unit='kg',
    description='CO2_e-Emissionen',
    maximum_per_hour=1000,  # Regulatory limit: max 1000 kg CO2/hour
)

## Define Components

Now we create the actual energy system components:

### Gas Boiler

A simple converter that transforms gas into heat:

In [6]:
boiler = fx.linear_converters.Boiler(
    label='Boiler',
    thermal_efficiency=0.5,  # 50% efficiency
    thermal_flow=fx.Flow(
        label='Q_th',
        bus='Fernwärme',
        size=50,  # Maximum capacity: 50 kW
        relative_minimum=0.1,  # Minimum part load: 10%
        relative_maximum=1,  # Maximum part load: 100%
    ),
    fuel_flow=fx.Flow(label='Q_fu', bus='Gas'),
)

### Combined Heat and Power (CHP)

A CHP unit generates both electricity and heat from fuel with fixed efficiency ratios:

In [7]:
chp = fx.linear_converters.CHP(
    label='CHP',
    thermal_efficiency=0.5,  # 50% thermal efficiency
    electrical_efficiency=0.4,  # 40% electrical efficiency
    electrical_flow=fx.Flow('P_el', bus='Strom', size=60, relative_minimum=5 / 60),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme'),
    fuel_flow=fx.Flow('Q_fu', bus='Gas'),
)

### Thermal Storage

Storage allows temporal decoupling of heat production and demand:

In [8]:
storage = fx.Storage(
    label='Storage',
    charging=fx.Flow('Q_th_load', bus='Fernwärme', size=1000),
    discharging=fx.Flow('Q_th_unload', bus='Fernwärme', size=1000),
    capacity_in_flow_hours=fx.InvestParameters(
        effects_of_investment=20,  # Investment cost: 20 €
        fixed_size=30,  # Fixed capacity: 30 kWh
        mandatory=True,  # Must be built
    ),
    initial_charge_state=0,  # Start empty
    relative_maximum_charge_state=1 / 100 * np.array([80, 70, 80, 80, 80, 80, 80, 80, 80]),
    relative_maximum_final_charge_state=0.8,
    eta_charge=0.9,  # 90% charging efficiency
    eta_discharge=1,  # 100% discharging efficiency
    relative_loss_per_hour=0.08,  # 8% loss per hour
    prevent_simultaneous_charge_and_discharge=True,
)

### Sinks and Sources

Complete the system with demand (sink) and supply (source) components:

In [9]:
# Heat demand - fixed profile
heat_sink = fx.Sink(
    label='Heat Demand',
    inputs=[fx.Flow(label='Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=heat_demand_per_h)],
)

# Gas supply with costs and emissions
gas_source = fx.Source(
    label='Gastarif',
    outputs=[
        fx.Flow(
            label='Q_Gas',
            bus='Gas',
            size=1000,
            effects_per_flow_hour={
                costs.label: 0.04,  # 0.04 €/kWh
                CO2.label: 0.3,  # 0.3 kg CO2/kWh
            },
        )
    ],
)

# Electricity feed-in (revenue)
power_sink = fx.Sink(
    label='Einspeisung',
    inputs=[
        fx.Flow(
            label='P_el',
            bus='Strom',
            effects_per_flow_hour=-1 * power_prices,  # Negative = revenue
        )
    ],
)

## Build the FlowSystem

Add all components and effects to the flow system:

In [10]:
flow_system.add_elements(costs, CO2, boiler, storage, chp, heat_sink, gas_source, power_sink)

2025-12-08 14:25:57.041 INFO     │ Registered new Effect: costs
2025-12-08 14:25:57.042 INFO     │ Registered new Effect: costs
2025-12-08 14:25:57.042 INFO     │ Registered new Effect: CO2
2025-12-08 14:25:57.042 INFO     │ Registered new Effect: CO2
2025-12-08 14:25:57.042 INFO     │ Registered new Boiler: Boiler
2025-12-08 14:25:57.043 INFO     │ Registered new Storage: Storage
2025-12-08 14:25:57.043 INFO     │ Registered new CHP: CHP
2025-12-08 14:25:57.043 INFO     │ Registered new Sink: Heat Demand
2025-12-08 14:25:57.043 INFO     │ Registered new Source: Gastarif
2025-12-08 14:25:57.044 INFO     │ Registered new Sink: Einspeisung


## Visualize the System Topology

Before optimizing, let's visualize the system structure to verify it's correct:

In [11]:
flow_system.topology.plot('topology.html', show=False)

<class 'pyvis.network.Network'> |N|=9 |E|=10

## Run the Optimization

Now we solve the optimization problem using the HiGHS solver:

In [12]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0, time_limit_seconds=30))

2025-12-08 14:25:57.314 INFO     │ Auto-created Penalty Effect: Penalty
2025-12-08 14:25:57.418 WARNING  │ ┌─ Flow Boiler(Q_th) has a relative_minimum of <xarray.DataArray 'Boiler(Q_th)|relative_minimum' (time: 9)> Size: 72B
                                 │ │  array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
                                 │ │  Coordinates:
                                 │ └─   * time     (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-01T08:00:00 and no status_parameters. This prevents the Flow from switching inactive (flow_rate = 0). Consider using status_parameters to allow the Flow to be switched active and inactive.
2025-12-08 14:25:57.586 WARNING  │ ┌─ Flow CHP(P_el) has a relative_minimum of <xarray.DataArray 'CHP(P_el)|relative_minimum' (time: 9)> Size: 72B
                                 │ │  array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
                                 │ │         0.08333333, 0.08333333, 0.08333333, 0.08333333]

FlowSystem
Timesteps: 9 (Hour) [2020-01-01 to 2020-01-01]
Periods: None
Scenarios: None
Status: ✓

Components (6 items)
--------------------
 * Boiler
 * CHP
 * Einspeisung
 * Gastarif
 * Heat Demand
 * Storage

Buses (3 items)
---------------
 * Fernwärme
 * Gas
 * Strom

Effects (3 items)
-----------------
 * CO2
 * costs
 * Penalty

Flows (10 items)
----------------
 * Boiler(Q_fu)
 * Boiler(Q_th)
 * CHP(P_el)
 * CHP(Q_fu)
 * CHP(Q_th)
 * Einspeisung(P_el)
 * Gastarif(Q_Gas)
 * Heat Demand(Q_th_Last)
 * Storage(Q_th_load)
 * Storage(Q_th_unload)

## Analyze Results

### Heat Balance

Let's see how the heat demand is met by different components:

In [13]:
flow_system.statistics.plot.balance('Fernwärme')

PlotResult(data=<xarray.Dataset> Size: 432B
Dimensions:                 (time: 9)
Coordinates:
  * time                    (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-...
Data variables:
    Boiler(Q_th)            (time) float64 72B -5.0 -5.0 -5.0 ... -5.0 -5.0 -5.0
    Storage(Q_th_unload)    (time) float64 72B 1.732e-14 1.348e-14 ... -0.0
    CHP(Q_th)               (time) float64 72B -25.0 -21.67 ... -15.0 -15.0
    Storage(Q_th_load)      (time) float64 72B -2.087e-14 26.67 ... 0.0
    Heat Demand(Q_th_Last)  (time) float64 72B 30.0 0.0 90.0 ... 20.0 20.0 20.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=Boiler(Q_th)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Boiler(Q_th)',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Boiler(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAFMAAAAAAAAAUwAAAAAAAAB' ... 'AAAAAAFMAAAAAAAAAUwAAAAAAAABTA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Storage(Q_th_unload)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Storage(Q_th_unload)',
              'marker': {'color': '#EF553B', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Storage(Q_th_unload)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('CgAAAACAEz1Ej8L1KFwOPdj//////y' ... 'AAAAAAAID+//////8RvQAAAAAAAACA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=CHP(Q_th)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'CHP(Q_th)',
              'marker': {'color': '#00CC96', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'CHP(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('////////OMCrqqqqqqo1wAAAAAAAwF' ... 'AAAAAALsABAAAAAAAuwAAAAAAAAC7A'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Storage(Q_th_load)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Storage(Q_th_load)',
              'marker': {'color': '#EF553B', 'line': {'width':

### Storage Behavior

Visualize how the storage charges and discharges:

In [14]:
flow_system.statistics.plot.balance('Storage')

PlotResult(data=<xarray.Dataset> Size: 216B
Dimensions:               (time: 9)
Coordinates:
  * time                  (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-01...
Data variables:
    Storage(Q_th_load)    (time) float64 72B 2.087e-14 -26.67 ... -0.0
    Storage(Q_th_unload)  (time) float64 72B -1.732e-14 -1.348e-14 ... 0.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=Storage(Q_th_load)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Storage(Q_th_load)',
              'marker': {'color': '#D62728', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Storage(Q_th_load)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('CgAAAACAFz2nqqqqqqo6wBzHcRzHcT' ... 'AAAAAAAID+//////8TvQAAAAAAAACA'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Storage(Q_th_unload)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Storage(Q_th_unload)',
              'marker': {'color': '#D62728', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Storage(Q_th_unload)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-01T01:00:00.000000000',
                          '2020-01-01T02:00:00.000000000', '2020-01-01T03:00:00.000000000',
                          '2020-01-01T04:00:00.000000000', '2020-01-01T05:00:00.000000000',
                          '2020-01-01T06:00:00.000000000', '2020-01-01T07:00:00.000000000',
                          '2020-01-01T08:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('CgAAAACAE71Ej8L1KFwOvdj//////y' ... 'AAAAAAAAD+//////8RPQAAAAAAAAAA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'bargap': 0,
               'bargroupgap': 0,
               'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Storage (flow_rate)'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

### Heatmap Visualizations

Heatmaps are useful for visualizing patterns over time:

In [15]:
flow_system.statistics.plot.heatmap('CHP(Q_th)')

PlotResult(data=<xarray.Dataset> Size: 152B
Dimensions:    (timeframe: 1, timestep: 9)
Coordinates:
  * timeframe  (timeframe) object 8B '2020-01-01'
  * timestep   (timestep) object 72B '00:00' '01:00' '02:00' ... '07:00' '08:00'
Data variables:
    value      (timestep, timeframe) float64 72B 25.0 21.67 75.0 ... 15.0 15.0, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>CHP(Q_th)|flow_rate: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2020-01-01'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00'], dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('////////OECrqqqqqqo1QAAAAAAAwF' ... 'AAAAAALkABAAAAAAAuQAAAAAAAAC5A'),
                    'dtype': 'f8',
                    'shape': '9, 1'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'CHP(Q_th)|flow_rate'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

In [16]:
flow_system.statistics.plot.heatmap('Storage')

PlotResult(data=<xarray.Dataset> Size: 152B
Dimensions:    (timeframe: 1, timestep: 9)
Coordinates:
  * timeframe  (timeframe) object 8B '2020-01-01'
  * timestep   (timestep) object 72B '00:00' '01:00' '02:00' ... '07:00' '08:00'
Data variables:
    value      (timestep, timeframe) float64 72B 0.0 -1.465e-15 24.0 ... 0.0 0.0, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>Storage|charge_state: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2020-01-01'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00'], dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAAAAAABTZmZmZmbavAAAAAAAAD' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'),
                    'dtype': 'f8',
                    'shape': '9, 1'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Storage|charge_state'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

### Access Raw Data

The results are available as xarray Datasets for further analysis:

In [17]:
print('Flow rates:')
flow_system.statistics.flow_rates

Flow rates:


<xarray.Dataset> Size: 792B
Dimensions:                 (time: 9)
Coordinates:
  * time                    (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-...
Data variables:
    Boiler(Q_fu)            (time) float64 72B 10.0 10.0 10.0 ... 10.0 10.0 10.0
    Boiler(Q_th)            (time) float64 72B 5.0 5.0 5.0 23.89 ... 5.0 5.0 5.0
    Storage(Q_th_load)      (time) float64 72B -2.087e-14 26.67 ... 0.0
    Storage(Q_th_unload)    (time) float64 72B -1.732e-14 -1.348e-14 ... 0.0
    CHP(Q_fu)               (time) float64 72B 50.0 43.33 150.0 ... 30.0 30.0
    CHP(Q_th)               (time) float64 72B 25.0 21.67 75.0 ... 15.0 15.0
    CHP(P_el)               (time) float64 72B 20.0 17.33 60.0 ... 12.0 12.0
    Heat Demand(Q_th_Last)  (time) float64 72B 30.0 0.0 90.0 ... 20.0 20.0 20.0
    Gastarif(Q_Gas)         (time) float64 72B 60.0 53.33 160.0 ... 40.0 40.0
    Einspeisung(P_el)       (time) float64 72B 20.0 17.33 60.0 ... 12.0 12.0

In [18]:
print('Charge states:')
flow_system.statistics.charge_states

Charge states:


<xarray.Dataset> Size: 144B
Dimensions:  (time: 9)
Coordinates:
  * time     (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-01T08:00:00
Data variables:
    Storage  (time) float64 72B 0.0 -1.465e-15 24.0 12.08 0.0 0.0 0.0 0.0 0.0

### Duration Curve

Duration curves show the sorted values, useful for sizing analysis:

In [19]:
flow_system.statistics.plot.duration_curve('Boiler(Q_th)')

PlotResult(data=<xarray.Dataset> Size: 144B
Dimensions:       (duration: 9)
Coordinates:
  * duration      (duration) int64 72B 0 1 2 3 4 5 6 7 8
Data variables:
    Boiler(Q_th)  (duration) float64 72B 35.0 23.89 5.0 5.0 5.0 5.0 5.0 5.0 5.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=Boiler(Q_th)<br>duration=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Boiler(Q_th)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Boiler(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'AAECAwQFBgcI', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAACAQUCkLEMc6+I3QAAAAAAAAB' ... 'AAAAAAFEAAAAAAAAAUQAAAAAAAABRA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Duration Curve'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Timesteps'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

### Effects Summary

View the temporal effects (costs, emissions) over time:

In [20]:
flow_system.statistics.temporal_effects

<xarray.Dataset> Size: 960B
Dimensions:         (contributor: 3, time: 9)
Coordinates:
  * contributor     (contributor) object 24B 'Einspeisung(P_el)' ... 'Storage'
  * time            (time) datetime64[ns] 72B 2020-01-01 ... 2020-01-01T08:00:00
    component       (contributor) <U11 132B 'Einspeisung' 'Gastarif' 'Storage'
    component_type  (contributor) <U7 84B 'Sink' 'Source' 'Storage'
Data variables:
    costs           (time, contributor) float64 216B -1.6 6.0 nan ... 4.0 nan
    CO2             (time, contributor) float64 216B nan 18.0 nan ... 12.0 nan
    Penalty         (time, contributor) float64 216B nan nan nan ... nan nan nan

## Summary

This example demonstrated:

- Creating a multi-carrier energy system (electricity, heat, gas)
- Using pre-built components (`Boiler`, `CHP`, `Storage`)
- Defining costs and emissions as effects
- Visualizing system topology and results
- Accessing results as xarray Datasets

For more advanced features, see the complex example and optimization modes notebooks!